# Projeto 1 - Classificador Naive-Bayes com Tweets


## Objetivo

* Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa selecionealgumas mensagens disponíveis no Twitter que mencionam esse produto e classifique-as como"relevante" ou "irrelevante".

* Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.

* Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamentedo Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails, porexemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.

* Para realizar o MVP (minimum viable product) do projeto, você precisa implementar umaversão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.

* Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Integrantes do Projeto

**Nome:** Wilgner Lopes Araujo

**Nome:** Fabricio Neri Lima

## Informações relevantes

* Esse arquivo representa a ultima etapa do projeto onde haverá todas as etapas de cleaning da base de dados e implantação do algoritmo de classificação

* O arquivo que pesquisa os tweets esta na pasta Arquivo Jupyter/Projeto1_Obtenção_dos_tweets nessa mesma raiz do repositório

* A base de dados se encontra na pasta dados/Valorant.xlsx

* A conta utiliza no twitter para obtenção de dados é @AnalyticsInsper

* O produto escolhido para o projeto é : **Valorant**

* Os criterios para a classificação dos tweets na primeira etapa do projeto foram: Os tweets que falavam de alguma forma positiva ou negativa sobre o jogo ou até mesmo que relacionavam o jogo a alguma emoção, foram classificados como relevantes. Os demais foram classificados como Irrelevantes.


# Começando

## Declarando as bibliotecas

In [1]:
#importando cada uma das bibliotecas que serão usadas
import math
import numpy as np
import pandas as pd
import re
import functools
import operator
import emoji

## Abrindo a base de dados e visualizando os primeiros 5 termos

In [2]:
#abrindo o arquivo de treino
dados = pd.read_excel('../dados/Valorant.xlsx')
dados.head()

,Treinamento,Etiquetas,Classificado
0,@caos__0 eu no valorant :v\nfon,2,Irrelevante
1,o cara q pra mim é o melhor player de valorant...,2,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante
3,"deletei o valorant, agora meu pc volta a ligar...",1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu...,1,Relevante


## Informações do DataFrame

In [3]:
#lendo as informações da base de treino
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Treinamento   500 non-null    object
 1   Etiquetas     500 non-null    int64 
 2   Classificado  500 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


## Classificando 

* A coluna Treinamento e Classificado são categoricas, pois ambas são variaveis qualitativas nominais

In [4]:
#definindo como tipo como variáveis categóricas
dados['Treinamento'] = dados['Treinamento'].astype('category')
dados['Classificado'] = dados['Classificado'].astype('category')

## Iniciando as coletas de probabilidades

### Analisando frequencia de Relevante e Irrelevante na base de dados

In [5]:
#identificando as probabilidades de Relevante P(R) e probabilidade de Irrelevante P(R_c)
Probabilidade_total = dados.Classificado.value_counts(True)
Probabilidade_total

Irrelevante    0.542
Relevante      0.458
Name: Classificado, dtype: float64

## Declarando as variaveis de probabilidade

* Considere P(R) para probabilidade de ser Relevante
* Considere P(R_c) para probabilidade de ser Irrelevante

In [6]:
#Probabilidade de Irrelevantes
P_R_c = Probabilidade_total[0]
#Probabilidade de relevantes
P_R = Probabilidade_total[1]

## Antes de prosseguir vamos limpar o codigo

* Será removido os seguintes simbolos: ! - . : ? ; _ , < > / \ ) ( # % $ | \n

In [7]:
#Função utilizada para realizar a limpeza no Dataframe
def limpa_texto(text):
    
    #import string
    punctuation = '[!-.:?;_,<>/\)(#%$|\n]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

## Alem disso vamos espaçar os emojis entre si

* O codigo utilizado foi extraido e modificado do seguinte site: https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python

* O codigo utiliza a biblioteca emoji, functools, operator para atingir o objetivo de separar os emojis

In [8]:
def separa_emoji(text):
    frase = text
    frase_split_emoji = emoji.get_emoji_regexp().split(frase)
    frase_split_com_espaço_em_branco = [substr.split() for substr in frase_split_emoji]
    frase_split = functools.reduce(operator.concat, frase_split_com_espaço_em_branco)
    frase_espacada = ' '.join(frase_split)
    return frase_espacada

## Aplicando a função de limpeza na base de dados

In [9]:
#Aplicando as funções no dataframe
dados['Treinamento'] = dados['Treinamento'].apply(limpa_texto)
dados['Treinamento'] = dados['Treinamento'].apply(separa_emoji)

In [10]:
# Checando se funcionou e se os emojis ainda funcionam
pd.set_option('max_colwidth', None)
dados.tail(10)

,Treinamento,Etiquetas,Classificado
490,vou dormir um pouco se vazar skin nova do valorant espero que meus amigos lindos me mandem,2,Irrelevante
491,@gabrielzurmely @thaliciaa me da skin no valorant que te ajudo com ela,2,Irrelevante
492,caraio eu fui jogar um valorant hj e foi so arrependimento n joguei porra nenhuma e pqp como eu to possesso de raiva,2,Irrelevante
493,@andre1tv player oldschool com disciplina e garra na mesma pegada joga muito e é dedicado como poucos qualquer time de valorant só tem a ganhar com ele quando ele tiver em time top br não vem reclamar depois tá bom dia https t co xbypqekvyu,2,Irrelevante
494,ontem subi a prata 1 no valorant e hj já subi pra prata 2 graças ao meu perfeito namorado ❤️,1,Relevante
495,rt @patochovsk 💥 sorteio de valorant points 💥 haverá 2 ganhadores ganhador 1 5550 valorant points ganhador 2 5550 valorant points 💥 quanto…,2,Irrelevante
496,@girafodott valorant tbm,2,Irrelevante
497,jogador de lol foi jogar valorant veja o que rolou,2,Irrelevante
498,alô valorant encontro partida nunca 20 minutos na fila kkkk,1,Relevante
499,quem vai jogar um valorant cmg hj,1,Relevante


## Preparação para fazer o smoothing

* Primeiro passo: Verificar a contagem de cada palavra no grupo de DataFrame Relevante e Irrelevante

In [11]:
# Separando o dataFrame em duas Strings com todas as palavras do grupo de Relevante e Irrelevante
dados_relevantes = dados[dados.Classificado == 'Relevante']
dados_irrelevantes = dados[dados.Classificado == 'Irrelevante']

# Transformando colunas em listas de palavras
palavras_relevantes = str(dados_relevantes.Treinamento.to_list()).split()
palavras_irrelevantes = str(dados_irrelevantes.Treinamento.to_list()).split()

# Contando aparição total das palavras
contagem_relevantes = pd.Series(palavras_relevantes).value_counts()
contagem_irrelevante = pd.Series(palavras_irrelevantes).value_counts()

# Somando total de contagens
N_relevante = contagem_relevantes.sum()
N_irrelevante = contagem_irrelevante.sum()




* Segundo passo: Aplicar o alfa nas contagem das palavras e o alfa x V no total de palavras 

In [12]:
#criando um alpha para realizar a técnica de smoothing
alfa = 1
#Criando um V para estimar o total de palavras da lingua 
V = 10e6

# Contagem + alfa
contagem_relevantes_com_alfa = contagem_relevantes + alfa
contagem_irrelevantes_com_alfa = contagem_irrelevante + alfa

# Total + alfaV
N_relevante_com_alfaV = N_relevante+alfa*V
N_irrelevante_com_alfaV = N_irrelevante+alfa*V


* Calculando probabilidade de cada palavra dada que é relevante e irrelevante



In [13]:
P_palavra_dado_relevante = contagem_relevantes_com_alfa/N_relevante_com_alfaV
P_palavra_dado_irrelevante = contagem_irrelevantes_com_alfa/N_irrelevante_com_alfaV

## O teorema de bayes requer as seguintes probabilidades

$P(Relevante|tweet)$:

$$P(Relevante|tweet) = \frac{P(tweet|Relevante) \cdot P(Relevante)}{P(tweet)}$$

Ou seja, precisamos de: 
- $P(tweet|Relevante)$: probabilidade de encontrar o tweet dado que ele é relevante;
- $P(tweet|Irrelevante)$: probabilidade de encontrar o tweet dado que ele é relevante;
- $P(Relevante)$:  probabilidade do tweet ser relevante;
- $P(Irrelevante)$: probabilidade do tweet ser irrelevante;
- $P(tweet)$: probabilidade de um tweet ocorrer na lingua portuguesa.


Para encontrar a probabilidade de $P(tweet|Relevante)$, temos que fazer:

$P(tweet | Relevante) = P(Palavra_1 | Relevante) \cdot P(Palavra_2 | Relevante) \cdot P(Palavra_3 | Relevante) \cdot ... P(Palavra_n | Relevante)$

Contudo, como as probabilidades são muito pequenas, basta fazer a soma dos expoentes. Logo, basta aplicar logaritmo:

$log(P(tweet | Relevante)) = 
log(P(Palavra_1 | Relevante)) + log(P(Palavra_2 | Relevante)) + ... log(P(Palavra_n | Relevante))$

Dessa forma, aplicando o log, temos que: 

$log(P(Relevante|tweet)) = \frac{log(P(Relevante|palavra_1)) + log(P(Relevante|palavra_2)) + log(P(Relevante|palavra_3)) + log(P(Relevante|palavra_4)) + log(P(Relevante))}{P(tweet)}$

Como iremos realizar a comparação entre $P(Relevante|Tweet)$ e $P(Irrelevante|Tweet)$, não é necessário se preocupar com a divisão ou a subtração pelo $P(Tweet)$, se aplicarmos $log$, pois esse termo se cancelará.

## Calculando a coluna de probabilidade relevante

In [33]:
#valor inicial
frase = 0

tabela_relevante = dict()

probabilidade_da_frase = 0

#para cada um dos índices da tabela de tweets
for indice in range(len(dados)):
    
    frase = dados.iloc[indice,0]
    #divide o tweet em uma lista de palavras que compõem o tweet
    palavras_da_frase = frase.split()
    
    for palavra in palavras_da_frase:
    #faz o cálculo da probabilidade de CADA palavra    
        
        if palavra in palavras_relevantes:
        #se for uma palavra conhecida entre as palavras presentes nos tweets relevantes

            probabilidade_da_palavra = P_palavra_dado_relevante[palavra]
            #como estamos apenas pegando os expoentes, estamos somando
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
            
        else:
        #se for uma palavra desconhecida entre a lista das palavras presentes nos tweets relevantes
        
            probabilidade_da_palavra = alfa/N_relevante_com_alfaV
            #como estamos apenas pegando os expoentes, estamos somando
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
    
    #adiciona a um dicionario em que a chave é o tweet e o item é a probabilidade dele
    tabela_relevante[frase] = (probabilidade_da_frase)
    probabilidade_da_frase = 0
    
df = pd.DataFrame.from_dict(data = tabela_relevante, orient='index').reset_index()
df.columns = ['Tweet', 'Probabilidade dado que é relevante']
df.head()

,Tweet,Probabilidade dado que é relevante
0,@caos 0 eu no valorant v fon,43.076253
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,87.415776
2,mano algm upa meu passe no valorant na humildade,55.373890
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,72.071868
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,118.716192


## Calculando a coluna de probabilidade irrelevante

In [15]:
frase = 0
tabela_irrelevante = dict()
probabilidade_da_frase = 0
for indice in range(len(dados)):
    frase = dados.iloc[indice,0]
    palavras_da_frase = frase.split()

    for palavra in palavras_da_frase:
        
        
        if palavra in palavras_irrelevantes:

            probabilidade_da_palavra = P_palavra_dado_irrelevante[palavra]
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
            
        else:
            
            probabilidade_da_palavra = alfa/N_irrelevante_com_alfaV
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
           
    tabela_irrelevante[frase] = (probabilidade_da_frase)
    probabilidade_da_frase = 0
    
df1 = pd.DataFrame.from_dict(data = tabela_irrelevante, orient='index').reset_index()
df1.columns = ['Tweet', 'Probabilidade dado que é irrelevante']
df1.head()

,Tweet,Probabilidade dado que é irrelevante
0,@caos 0 eu no valorant v fon,41.436916
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,84.870098
2,mano algm upa meu passe no valorant na humildade,54.365370
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,72.956548
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,119.731469


## Juntando resultados

In [16]:
novodf = pd.concat([dados, df['Probabilidade dado que é relevante'], df1['Probabilidade dado que é irrelevante']], axis=1)

In [17]:
novodf.head()

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante
0,@caos 0 eu no valorant v fon,2,Irrelevante,43.076253,41.436916
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,87.415776,84.870098
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,55.373890,54.365370
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,72.071868,72.956548
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,118.716192,119.731469


## Aplicando calculo final 

In [18]:
novodf['Probabilidade dado relevante x Probabilidade de Relevante'] = novodf['Probabilidade dado que é relevante']+math.log10(P_R)
novodf['Probabilidade dado irrelevante x Probabilidade de Irelevante'] = novodf['Probabilidade dado que é irrelevante']+math.log10(P_R_c)

In [19]:
novodf.head(10)

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante,Probabilidade dado relevante x Probabilidade de Relevante,Probabilidade dado irrelevante x Probabilidade de Irelevante
0,@caos 0 eu no valorant v fon,2,Irrelevante,43.076253,41.436916,42.737119,41.170915
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,87.415776,84.870098,87.076642,84.604098
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,55.373890,54.365370,55.034755,54.099369
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,72.071868,72.956548,71.732734,72.690547
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,118.716192,119.731469,118.377058,119.465468
5,voltei a jogar valorant so por causa do mata mata,1,Relevante,58.942680,59.773108,58.603545,59.507107
6,viciei em valorant esse é o tweet,1,Relevante,41.539170,40.444920,41.200036,40.178919
7,@zncabral free fire fez mais sucesso que o valorant só no brasil no resto do mundo quase ngm joga isso,1,Relevante,121.148123,121.552353,120.808988,121.286352
8,alguem me explica o motivo das 12 do valorant menos a shorty atirar tão bem de longe tipo comparando com cs e fortnite não faz sentido os cara vai de bucky atacando vai se fuder,1,Relevante,213.539203,213.133537,213.200069,212.867536
9,rt @leozjn o matchmaking do valorant n faz o minimo sentido,1,Relevante,66.497866,66.064682,66.158731,65.798682


## Aplicando:

Se $P(Relevante|tweet) < P(Irrelevante|tweet)$, classificaremos o tweet como Relevante (Como estamos verificando o expoente das probabilidades e eles são números muito pequenos (expoente negativo), queremos o expoente mais próximo de 0), senão, classificaremos como Irrelevante.

In [20]:
novodf['Navy Bayes'] = np.where(novodf['Probabilidade dado que é relevante'] < novodf['Probabilidade dado que é irrelevante'], 'Relevante', 'Irrelevante')
novodf.head()

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante,Probabilidade dado relevante x Probabilidade de Relevante,Probabilidade dado irrelevante x Probabilidade de Irelevante,Navy Bayes
0,@caos 0 eu no valorant v fon,2,Irrelevante,43.076253,41.436916,42.737119,41.170915,Irrelevante
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,87.415776,84.870098,87.076642,84.604098,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,55.373890,54.365370,55.034755,54.099369,Irrelevante
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,72.071868,72.956548,71.732734,72.690547,Relevante
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,118.716192,119.731469,118.377058,119.465468,Relevante


In [21]:
tabela = pd.crosstab(novodf.Classificado, novodf['Navy Bayes'], normalize=True)
tabela

Navy Bayes,Irrelevante,Relevante
Classificado,,
Irrelevante,0.538,0.004
Relevante,0.174,0.284


In [22]:
precisao = tabela.iloc[0,0] + tabela.iloc[1,1]
precisao

0.8220000000000001

# Teste

In [23]:
teste = pd.read_excel('../dados/Valorant.xlsx', sheet_name = 'Teste')
teste.head()

,Teste,Etiquetas,Classificado
0,"mt silverio abandona a carreira como dj e vira pro player de valorant, em breve nos cinemas",2,Irrelevante
1,"definitivamente o valorant n quer mais me ver jogar, é cada bronze com jogatina de imortal q pelamor",1,Relevante
2,eu jogando valorant com meus amigos // eu jogando valorant sozinho https://t.co/esjhdslejb,2,Irrelevante
3,eu sei que a minha equipa no valorant é má quando sou eu a top fragger,2,Irrelevante
4,baixei valorant.. esse foi o erro da noite kkk,1,Relevante


### Checa relevante

In [24]:
frase = 0
tabela_relevante = dict()
probabilidade_da_frase = 0
for indice in range(len(teste)):
    frase = teste.iloc[indice,0]
    palavras_da_frase = frase.split()

    for palavra in palavras_da_frase:
        
        
        if palavra in palavras_relevantes:

            probabilidade_da_palavra = P_palavra_dado_relevante[palavra]
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
            
        else:
            
            probabilidade_da_palavra = alfa/N_relevante_com_alfaV
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
     
    tabela_relevante[frase] = (probabilidade_da_frase)
    probabilidade_da_frase = 0
    
df = pd.DataFrame.from_dict(data = tabela_relevante, orient='index').reset_index()
df.columns = ['Tweet', 'Probabilidade dado que é relevante']
df.head()

,Tweet,Probabilidade dado que é relevante
0,"mt silverio abandona a carreira como dj e vira pro player de valorant, em breve nos cinemas",107.851813
1,"definitivamente o valorant n quer mais me ver jogar, é cada bronze com jogatina de imortal q pelamor",109.700044
2,eu jogando valorant com meus amigos // eu jogando valorant sozinho https://t.co/esjhdslejb,70.892549
3,eu sei que a minha equipa no valorant é má quando sou eu a top fragger,93.703128
4,baixei valorant.. esse foi o erro da noite kkk,57.298812


### Checa Irrelevante

In [25]:
frase = 0
tabela_irrelevante = dict()
probabilidade_da_frase = 0
for indice in range(len(teste)):
    frase = teste.iloc[indice,0]
    palavras_da_frase = frase.split()

    for palavra in palavras_da_frase:
        
        
        if palavra in palavras_irrelevantes:

            probabilidade_da_palavra = P_palavra_dado_irrelevante[palavra]
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
            
        else:
            
            probabilidade_da_palavra = alfa/N_irrelevante_com_alfaV
            probabilidade_da_frase += abs(math.log10(probabilidade_da_palavra))
           
    tabela_irrelevante[frase] = (probabilidade_da_frase)
    probabilidade_da_frase = 0
    
df1 = pd.DataFrame.from_dict(data = tabela_irrelevante, orient='index').reset_index()
df1.columns = ['Tweet', 'Probabilidade dado que é irrelevante']
df1.head()

,Tweet,Probabilidade dado que é irrelevante
0,"mt silverio abandona a carreira como dj e vira pro player de valorant, em breve nos cinemas",105.361818
1,"definitivamente o valorant n quer mais me ver jogar, é cada bronze com jogatina de imortal q pelamor",108.597595
2,eu jogando valorant com meus amigos // eu jogando valorant sozinho https://t.co/esjhdslejb,70.352071
3,eu sei que a minha equipa no valorant é má quando sou eu a top fragger,93.007725
4,baixei valorant.. esse foi o erro da noite kkk,57.061789


### Junta resultados

In [26]:
novodf = pd.concat([dados, df['Probabilidade dado que é relevante'], df1['Probabilidade dado que é irrelevante']], axis=1)
novodf.head()

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante
0,@caos 0 eu no valorant v fon,2,Irrelevante,107.851813,105.361818
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,109.700044,108.597595
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,70.892549,70.352071
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,93.703128,93.007725
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,57.298812,57.061789


### Gera coluna final para analise

In [27]:
novodf['Probabilidade dado relevante x Probabilidade de Relevante'] = novodf['Probabilidade dado que é relevante']+math.log10(P_R)
novodf['Probabilidade dado irrelevante x Probabilidade de Irelevante'] = novodf['Probabilidade dado que é irrelevante']+math.log10(P_R_c)
novodf.head()

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante,Probabilidade dado relevante x Probabilidade de Relevante,Probabilidade dado irrelevante x Probabilidade de Irelevante
0,@caos 0 eu no valorant v fon,2,Irrelevante,107.851813,105.361818,107.512678,105.095817
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,109.700044,108.597595,109.360909,108.331594
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,70.892549,70.352071,70.553414,70.086070
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,93.703128,93.007725,93.363994,92.741724
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,57.298812,57.061789,56.959678,56.795788


### Aplica Navy-Bayes

In [28]:
novodf['Navy Bayes'] = np.where(novodf['Probabilidade dado que é relevante'] < novodf['Probabilidade dado que é irrelevante'], 'Relevante', 'Irrelevante')
novodf.head()

,Treinamento,Etiquetas,Classificado,Probabilidade dado que é relevante,Probabilidade dado que é irrelevante,Probabilidade dado relevante x Probabilidade de Relevante,Probabilidade dado irrelevante x Probabilidade de Irelevante,Navy Bayes
0,@caos 0 eu no valorant v fon,2,Irrelevante,107.851813,105.361818,107.512678,105.095817,Irrelevante
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante,109.700044,108.597595,109.360909,108.331594,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante,70.892549,70.352071,70.553414,70.086070,Irrelevante
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante,93.703128,93.007725,93.363994,92.741724,Irrelevante
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante,57.298812,57.061789,56.959678,56.795788,Irrelevante


In [29]:
tabela = pd.crosstab(novodf.Classificado, novodf['Navy Bayes'], normalize=True)
tabela

Navy Bayes,Irrelevante,Relevante
Classificado,,
Irrelevante,0.536,0.006
Relevante,0.450,0.008


In [30]:
precisao = tabela.iloc[0,0] + tabela.iloc[1,1]
precisao

0.544